# Multi-agent application

This notebook is an adaptation of the original LangGraph notebook: https://langchain-ai.github.io/langgraph/tutorials/multi_agent/hierarchical_agent_teams/

In [1]:
%pip install -U langgraph

   ---------------------------------------- 0.0/145.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/145.2 kB ? eta -:--:--
   -------- ------------------------------ 30.7/145.2 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------  143.4/145.2 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 145.2/145.2 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 0.1.7
    Uninstalling langgraph-prebuilt-0.1.7:
      Successfully uninstalled langgraph-prebuilt-0.1.7
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.3.20
    Uninstalling langgraph-0.3.20:
      Successfully uninstalled langgraph-0.3.20
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

env_path = r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\ODSC May 2025\multi-agent-langgraph"
load_dotenv(dotenv_path=env_path, override=True)

# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
tavily_api_key = os.getenv("TAVILY_API_KEY")
alphavantage_api_key = os.getenv("ALPHAVANTAGE_API_KEY")


In [2]:
# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = llm.invoke([message])

# Print the response
print(response)

content="J'adore programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BSRQGG6e93m00cIai8pENEYRusZmi', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'det

## Research Agent

In [3]:
import json
import random
from datetime import datetime

# Sample list of stock symbols and sectors
assets = [
    {"symbol": "AAPL", "sector": "Technology"},
    {"symbol": "GOOGL", "sector": "Technology"},
    {"symbol": "MSFT", "sector": "Technology"},
    {"symbol": "AMZN", "sector": "Consumer Discretionary"},
    {"symbol": "TSLA", "sector": "Consumer Discretionary"},
    {"symbol": "JNJ", "sector": "Healthcare"},
    {"symbol": "NVDA", "sector": "Technology"},
    {"symbol": "XOM", "sector": "Energy"},
    {"symbol": "META", "sector": "Communication Services"},
    {"symbol": "V", "sector": "Financials"},
    {"symbol": "PG", "sector": "Consumer Staples"},
    {"symbol": "BABA", "sector": "Consumer Discretionary"},
    {"symbol": "JPM", "sector": "Financials"},
    {"symbol": "KO", "sector": "Consumer Staples"},
    {"symbol": "DIS", "sector": "Communication Services"},
    {"symbol": "PFE", "sector": "Healthcare"},
]

def generate_portfolio(assets):
    portfolio = []
    for asset in assets:
        quantity = random.randint(5, 100)
        purchase_price = round(random.uniform(50, 3000), 2)
        total_invested = round(quantity * purchase_price, 2)
        purchase_date = datetime.strftime(datetime(2022, random.randint(1,12), random.randint(1,28)), "%Y-%m-%d")

        portfolio.append({
            "symbol": asset["symbol"],
            "sector": asset["sector"],
            "quantity": quantity,
            "purchase_price": purchase_price,
            "total_invested": total_invested,
            "purchase_date": purchase_date
        })
    return portfolio

# Generate and save portfolio
portfolio_data = generate_portfolio(assets)

with open("sample_portfolio.json", "w") as f:
    json.dump(portfolio_data, f, indent=4)

print("✅ Portfolio JSON saved to 'sample_portfolio.json'")


✅ Portfolio JSON saved to 'sample_portfolio.json'


### Web tool

In [3]:
from typing import Annotated, List

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
# Load environment variables from .env file
load_dotenv()

tavily_tool = TavilySearchResults(max_results=5, tavily_api_key=tavily_api_key)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages


class State(MessagesState):
    next: str


def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    class Router(TypedDict):
        """Worker to route to next. If no workers needed, route to FINISH."""

        next: Literal[*options]

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node

In [16]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict

search_agent = create_react_agent(llm, tools=[tavily_tool])


def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )





# Portfolio analysis Team

### Portfolio read tool

In [7]:
import json

@tool
def read_sample_portfolio(json_path: str = "sample_portfolio.json") -> str:
    """
    Reads the sample_portfolio.json file and returns its content as a string.
    Each entry includes the stock symbol, sector, quantity, purchase price, and purchase date.
    """
    if not os.path.exists(json_path):
        return f"File not found: {json_path}"

    with open(json_path, "r") as f:
        portfolio = json.load(f)

    if not isinstance(portfolio, list):
        return "Unexpected portfolio format."

    response = "Sample Portfolio:\n"
    for stock in portfolio:
        response += (
            f"- {stock['symbol']} ({stock['sector']}): "
            f"{stock['quantity']} shares @ ${stock['purchase_price']} "
            f"(Bought on {stock['purchase_date']})\n"
        )
    return response

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict


read_portfolio_agent = create_react_agent(llm, tools=[read_sample_portfolio])


def read_portfolio_node(state: State) -> Command[Literal["supervisor"]]:
    result = read_portfolio_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="read_portfolio")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )




# Report generator Agent

In [10]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional
from typing_extensions import TypedDict

# Define a real directory path
REAL_DIRECTORY = Path(r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\ODSC May 2025\multi-agent-langgraph\output")

#_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(REAL_DIRECTORY)

In [11]:
@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"



In [13]:
report_prompt = """

You are an expert report generator. Given the input from other agents, you generate a detailed report on how to optimize the provided portfolio.
The report will have the following outline:

-------------------------------
**Introduction on market landscape**
**Portfolio Overview**
**Investment Strategy**
**Performance Analysis**
**Recommendations**
**Conclusion**

**References**
--------------------------------

Once generated the report, save it using your write_document tool.


"""

doc_writer_agent = create_react_agent(
    llm,
    tools=[write_document],
    prompt=report_prompt,
)


def doc_writing_node(state: State) -> Command[Literal["supervisor"]]:
    result = doc_writer_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="doc_writer")
            ]
        },
        # We want our workers to ALWAYS "report back" to the supervisor when done
        goto="supervisor",
    )




# All together

In [14]:
from langchain_core.messages import BaseMessage

supervisor_node = make_supervisor_node(llm, ["search","read_portfolio", "doc_writer"])


In [17]:
# Define the graph.
builder = StateGraph(State)
builder.add_node("supervisor", supervisor_node)
builder.add_node("read_portfolio", read_portfolio_node)
builder.add_node("search", search_node)
builder.add_node("doc_writer", doc_writing_node)

builder.add_edge(START, "supervisor")
super_graph = builder.compile()

In [19]:
for s in super_graph.stream(
    {
        "messages": [
            ("user", "Generate a well structured report on how to improve my portfolio given the market landscape in Q4 2025.")
        ],
    },
    {"recursion_limit": 150},
):
    print(s)
    print("---")

{'supervisor': {'next': 'search'}}
---
{'search': {'messages': [HumanMessage(content='**Portfolio Improvement Report Based on Market Landscape Q4 2025**\n\n---\n\n### 1. **Market Landscape Highlights for Q4 2025:**\nThe market trends observed for Q4 2025 suggest:\n- **Elevated Interest Rates:** Policy rates in developed markets, particularly in the US, are expected to remain high. This "higher-for-longer" rate narrative creates opportunities for varied asset positioning.\n- **Equity Market Dispersion:** Stock performance varies greatly across sectors, countries, and themes, reflecting significant dispersion. The S&P 500 shows an optimistic price target with selective stock opportunities.\n- **Global Economy:** Robust growth overall, with a slowdown in China, and easing consumption levels in the US by late 2025.\n- **Sector Opportunities:** Emerging markets and specific growth sectors (e.g., technology, healthcare) hold selective promise, though country-specific and risk factors should 